In [3]:
using DataFrames
using LowRankModels


# Pull command line arguments
data_directory = ARGS[1]
k = parse(Int64, ARGS[2])
nfolds = parse(Int64, ARGS[3])
@show data_directory, k, nfolds

# Read in training data
df = readtable(string(data_directory, "/all_samples_filtered2_train.csv"), header=false)
m, n = size(df)

# Form sparse array
all_data = sparse(Array(df))
dropzeros!(all_data)
p = size(nonzeros(all_data), 1)

# Offset values so that 0 indicates missing and all other entries are 1, 2, 3, etc
all_data[all_data .> 0] = all_data[all_data .> 0] + 1
all_data[all_data.==-1] = 1

# Pull observed (nonzero) entries
i, j, v = findnz(all_data)
obs = collect(zip(i, j))

@show maximum(all_data), minimum(all_data), size(obs, 1)

INFO: Precompiling module LowRankModels.

Use "abstract type Domain end" instead.

Use "abstract type Loss end" instead.

Use "abstract type DiffLoss<:Loss end" instead.

Use "abstract type ClassificationLoss<:Loss end" instead.

Use "abstract type Regularizer end" instead.

Use "abstract type MatrixRegularizer<:LowRankModels.Regularizer end" instead.
This may mean module ForwardDiff does not support precompilation but is imported by a module that does.
ERROR: LoadError: Declaring __precompile__(false) is not allowed in files that are being precompiled.
Stacktrace:
 [1] _require(::Symbol) at ./loading.jl:455
 [2] require(::Symbol) at ./loading.jl:405
 [3] include_from_node1(::String) at ./loading.jl:576
 [4] include(::String) at ./sysimg.jl:14
 [5] anonymous at ./<missing>:2
while loading /Users/kelley/.julia/v0.6/Roots/src/Roots.jl, in expression starting on line 11
ERROR: LoadError: LoadError: Failed to precompile Roots to /Users/kelley/.julia/lib/v0.6/Roots.ji.
Stacktrace:
 [1] comp

LoadError: [91mFailed to precompile LowRankModels to /Users/kelley/.julia/lib/v0.6/LowRankModels.ji.[39m

In [4]:
Pkg.status() 

9 required packages:
 - ArrayViews                    0.6.4
 - DataFrames                    0.10.1
 - FactCheck                     0.4.3
 - IJulia                        1.7.0
 - LowRankModels                 0.2.0
 - MLBase                        0.7.0
 - NullableArrays                0.1.2
 - Plots                         0.15.0
 - PyCall                        1.15.0+            master
46 additional packages:
 - BinDeps                       0.8.6
 - Calculus                      0.2.2
 - ColorTypes                    0.6.7
 - Colors                        0.8.2
 - CommonSubexpressions          0.0.1
 - Compat                        0.54.0
 - Conda                         0.7.1
 - Contour                       0.4.0
 - DataArrays                    0.6.2
 - DataStructures                0.7.4
 - DiffResults                   0.0.3
 - DiffRules                     0.0.3
 - FileIO                        0.7.0
 - FixedPointNumbers             0.4.6
 - ForwardDiff                   0.

In [6]:
Pkg.update()

INFO: Updating METADATA...
INFO: Updating cache of CommonSubexpressions...
INFO: Updating cache of ColorTypes...
INFO: Updating cache of Contour...
INFO: Updating cache of PlotUtils...
INFO: Updating cache of Plots...
INFO: Updating cache of Requires...
INFO: Updating cache of ArrayViews...
INFO: Updating cache of Measures...
INFO: Updating cache of MbedTLS...
INFO: Updating cache of BinDeps...
INFO: Updating cache of IterTools...
INFO: Updating cache of ZMQ...
INFO: Updating cache of IJulia...
INFO: Updating cache of StatsBase...
INFO: Updating cache of MacroTools...
INFO: Updating cache of Nullables...
INFO: Updating cache of URIParser...
INFO: Updating cache of LineSearches...
INFO: Updating cache of Roots...
INFO: Updating cache of DataFrames...
INFO: Updating cache of DiffRules...
INFO: Updating cache of ForwardDiff...
INFO: Updating cache of Conda...
INFO: Updating cache of RecipesBase...
INFO: Updating cache of FixedPointNumbers...
INFO: Updating cache of FileIO...
INFO: Updatin

[21:58:38] ######################################################################### 100.0%[21:58:38] ##                                                                          3.6%[21:58:38] #########                                                                  12.8%[21:58:38] ########################                                                   33.1%[21:58:38] #######################################################                    75.5%[21:58:38] #########################################################                  79.2%[21:58:38] ###############################################################            86.4%


Info: using prebuilt binaries
INFO: Building ZMQ
Info: Downloading https://github.com/JuliaInterop/ZMQBuilder/releases/download/v4.2.5+6/ZMQ.x86_64-apple-darwin14.tar.gz to /Users/kelley/.julia/v0.6/ZMQ/deps/usr/downloads/ZMQ.x86_64-apple-darwin14.tar.gz...


[21:58:50] ######################################################################### 100.0%[21:58:46]                                                                             0.2%[21:58:46]                                                                             0.7%[21:58:46] #                                                                           2.1%[21:58:46] ####                                                                        5.9%[21:58:46] ########                                                                   11.8%[21:58:46] ################                                                           22.1%[21:58:47] ###################                                                        27.1%[21:58:47] ####################                                                       28.7%[21:58:47] #########################                                                  34.6%[21:58:47] ############################                                               38.4

INFO: Building IJulia
INFO: Installing Jupyter via the Conda package.


...



==> WARNING: A newer version of conda exists. <==
  current version: 4.4.10
  latest version: 4.5.8

Please update conda by running

    $ conda update -n base conda


openssl 1.0.2o: ########## | 100% 
ca-certificates 2018.03.07: ########## | 100% 
certifi 2018.4.16: 8          |   8% 


## Package Plan ##

  environment location: /Users/kelley/.julia/v0.6/Conda/deps/usr

  added / updated specs: 
    - jupyter


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    openssl-1.0.2o             |       h26aff7b_0         3.4 MB
    ca-certificates-2018.03.07 |                0         124 KB
    certifi-2018.4.16          |           py27_0         142 KB
    ------------------------------------------------------------
                                           Total:         3.7 MB

The following packages will be UPDATED:

    ca-certificates: 2017.08.26-ha1e5d58_0      --> 2018.03.07-0     
    certifi:         2017.7.27.1-py27h482ffc0_0 --> 2018.4.16-py27_0 
    openssl:         1.0.2l-h57f3a61_2          --> 1.0.2o-h26aff7b_0


Preparing transaction: ...working... 

certifi 2018.4.16: ########## | 100% 


done
Verifying transaction: ...working... done
Executing transaction: ...working... done


INFO: Found Jupyter version 4.4.0: /Users/kelley/.julia/v0.6/Conda/deps/usr/bin/jupyter
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 12309  100 12309    0     0  63302      0 --:--:-- --:--:-- --:--:-- 67262
INFO: Installing Julia kernelspec julia-0.6
[InstallKernelSpec] Removing existing kernelspec in /Users/kelley/Library/Jupyter/kernels/julia-0.6
[InstallKernelSpec] Installed kernelspec julia-0.6 in /Users/kelley/Library/Jupyter/kernels/julia-0.6
INFO: Building Plots
